___

In [2]:
# Imports
#from pyspark.sql import SparkSession
#import pyspark.sql.functions as f

ModuleNotFoundError: No module named 'pyspark'

# (I can't get no) satisfaction
Wir arbeiten weiterhin mit dem Datensatz über die Zufriedenheit von Angestellten einer fiktiven Firma. In dieser Kurseinheit betrachten wir die parallele Verarbeitung mit Spark. 

*Anmerkung: Der verwendete Datensatz ist natürlich eher klein, so dass normalerweise keine Parallelisierung benötigt wird. In der Kurseinheit dient er aber dazu, die grundlegenden Konzepte von Spark zu vermitteln.*

*Wichtige Methoden finden Sie im Foliensatz, es empfiehlt sich jedoch auch, die <a href="https://spark.apache.org/docs/latest/api/python/reference/index.html">pyspark-Doku</a> für die Bearbeitung zu nutzen.*

*In dieser Kurseinheit werden u.a. Lambda-Funktionen verwendet. Wenn Sie damit noch nicht vetraut sind, hilft ein Blick in die <a href="https://www.w3schools.com/python/python_lambda.asp">Python-Lambas-Doku</a>.*

Den Eintrittspunkt in die Funktionalität von Spark stellt der SparkContext dar. Dieser repräsentiert auch die Verbindung zu den Exekutoren. Bei der Erstellung wird außerdem eine Sammlung von Konfigurationsparametern mitgegeben. Wir beschränken uns zunächst auf die Angabe des Masters und einem App-Namen:
- **master:** Wir verwenden YARN als Ressourcen-Manager und starten die Anwendung im Client-Modus.
- **appName:** Hier vergeben wir einen Namen für unsere Applikation.

Da wir später auch mit SparkSQL arbeiten wollen, erstellen wir zudem direkt eine SparkSession. Diese stellt den Eingangspunkt für die SparkSQL-Funktionalität zur Verfügung und beinhaltet auch den SparkContext.

In [ ]:
# SparkSession erstellen
spark = 

# SparkContext
sc = spark.sparkContext

## RDDs
Zunächst betrachten wir die Arbeit mit RDDs. Wie Sie in den Vorlesungsvideos gelernt haben, bilden diese die Grundlage der parallelen Verarbeitung. In der Praxis wird meist nicht direkt mit RDDs, sondern mit einer High-level-Bibliothek wie SparkSQL gearbeitet. Es ist jedoch wichtig, die Konzepte von RDDs zu verstehen, da diese die Basis von Spark darstellen. 

### Eigenschaften von RDDs
Betrachten wir zunächst die wichtigsten Eigenschaften von RDDs:
- In-Memory-Verarbeitung
- Lazy
- Partitioniert
- Unveränderlich
- Lineage

#### In-Memory-Verarbeitung
Beschreiben Sie die Vorteile der In-Memory-Verarbeitung.

#### Lazy
Beschreiben Sie, was es für die Programmierung bedeutet, dass die Verarbeitung lazy ausgeführt wird. Programmieren Sie ein Code-Beispiel Ihrer Wahl, in dem Sie die Auswirkungen der lazy Ausführung demonstrieren. Nennen Sie zudem die wichtigsten Unterschiede zwischen Transformations und Actions.

In [ ]:
# Ihr Beispiel für eine lazy Ausführung

#### Partitioniert
Erstellen Sie aus der vorgegebenen Liste (`data`) ein RDD namens `rdd_part` mit drei Partitionen. Kontrollieren Sie nach der Erstellung, dass die korrekte Anzahl Partitionen erzeugt wurde.

In [ ]:
data = [1 for i in range (10)] + [0 for i in range (5)]

In [ ]:
# Ihre Lösung

In [ ]:
# RDD erstellen
rdd_part =

In [ ]:
# Anzahl Partitionen anzeigen

Im Folgenden schauen wir uns die Verteilung der Daten auf die unterschiedlichen Partitionen an.

> **Achtung:** Die Anwendung von `collect()` in Kombination mit `glom()` ist nur für kleine Datensätze geeignet!

In [ ]:
rdd_part.glom().collect()

Wie wir sehen können, bestehen zwei Partitionen aus 1en und eine aus 0en. Filtern Sie das RDD nun so, dass nur die Werte größer 0 übrig bleiben. Erstellen Sie hierfür ein neues RDD namens `rdd_filtered`.

In [ ]:
# Ihre Lösung
rdd_filtered =

Wenn wir uns nun die Verteilung der Daten auf die Partitionen ansehen, sehen wir, dass wir eine leere Partition erzeugt haben:

In [ ]:
rdd_filtered.glom().collect()

Beschreiben Sie die Nachteile von leeren Partitionen. Wie kann dieses Problem gelöst werden (Stichwort *Shuffling*) und warum sollte Shuffling normalerweise vermieden werden?

#### Unveränderlich und Lineage
Warum sind RDDs unveränderlich und warum kennt jedes RDD seine Abstammung (Lineage)?

### Daten laden
Laden Sie den Datensatz `employees_satisfaction_transformed.csv` in ein RDD. Denken Sie daran, dass die Daten standardmäßig aus dem HDFS geladen werden. Aus diesem Grund müssen die Daten zunächst ins HDFS geladen werden. Verwenden Sie hierzu das Notebook `hdfs-upload.ipynb`, das wir Ihnen in Moodle zur Verfügung gestellt haben.

Laden Sie anschließend die Daten in ein RDD.

In [ ]:
# Ihre Lösung

### Erste Datenexploration

Lassen Sie sich dann die ersten zehn Zeilen des RDDs anzeigen.

In [ ]:
# Ihre Lösung

Die erste Zeile entspricht dem Header. Entfernen Sie diese aus dem Datensatz.

In [ ]:
# Ihre Lösung

Zählen Sie nun die Anzahl der Angestellten.

In [ ]:
# Ihre Lösung

### Datenverarbeitung

Zählen Sie nun die Angestellten pro Department.

*Sollten Sie keinen Lösungsansatz haben, finden Sie unter dem Button "Show Solution" die Beschreibung der notwendigen Verarbeitungs-Schritte.*

1. Aktuell ist jede Zeile des RDD ein String, in dem die einzelnen Felder durch Komma getrennt sind. Dieser String muss zunächst gesplittet werden.
2. Im nächsten Schritt müssen Key-Value-Paare gebildet werden. Der Key ist das Department und der Wert ist 1.
3. Dann müssen pro Key alle Werte addiert werden.

*Wenn Sie Probleme mit der konkreten Umsetzung der Aufgabe haben, finden Sie unter dem Button "Show Solution" Hinweise zur Programmierung.*

- Das Splitten der einzelnen Funktionen erfolgt mit `split(",")`.
- Anschließend liegen die Daten in einer Liste vor. Das dritte Element (`line[2]`) entspricht dem Department.
- Mithilfe der Funktion `reducyByKey()` können die Werte pro Key addiert werden. 

In [ ]:
# Ihre Lösung

Die Arbeit mit RDDs ist offensichtlich recht mühsam, weshalb in der Praxis häufig eine High-level API wie SparkSQL verwendet wird. Diese schauen wir uns im Folgenden an.

## DataFrames
Im Folgenden arbeiten wir mit der High-level API SparkSQL.

*Vorsicht: DataFrames sind die zentrale Abstraktion der SparkSQL-Bibiliothek. Die Bibliothek bietet zwei praktisch gleichwertige APIs, um mit DataFrames zu arbeiten: Die **DataFrame API** und die **SQL API**. Wir arbeiten zunächst mit der DataFrame API.*

### Daten laden
Laden Sie die Daten aus der Datei `employees_satisfaction_transformed.csv` in ein DataFrame. Die Spaltenbezeichnungen sollen dabei im Header stehen und nicht Teil der Daten sein, nutzen Sie außerdem die automatische Schemaerkennung von Spark.

In [ ]:
# Ihre Lösung

Lassen Sie sich das Schema ausgeben.

In [ ]:
# Ihre Lösung

Lassen Sie sich die Daten anzeigen.

In [ ]:
# Ihre Lösung

Geben Sie die Anzahl Angestellter aus.

In [ ]:
# Ihre Lösung

DataFrames basieren auf RDDs. Deshalb kann auch jedes DataFrame in ein RDD umgewandelt werden.

Erstellen Sie aus dem DataFrame der Angestellten ein RDD und lassen Sie sich die ersten fünf Zeilen anzeigen.

In [ ]:
# Ihre Lösung

Das RDD besteht aus einzelnen Rows. Mittels Index kann auf die einzelnen Rows zugegriffen werden. Der Zugriff auf die Spalten erfolgt entweder auch durch die Angabe des Index oder durch den Spaltennamen.

Lassen Sie sich das Gehalt der ersten Row ausgeben.

In [ ]:
# Ihre Lösung

### Verarbeitung
*Hinweis: Für die Bearbeitung der folgenden Aufaben ist es hilfreich, sich die `pyspark.sql.functions` anzusehen. Diese sind bereits importiert und können mit `f.<function_name>` verwendet werden.*

Fügen Sie eine neue Spalte `service_days` hinzu, in der angegeben wird, seit wie vielen Tagen ein\*e Angestellte\*r im Unternehmen ist. Verwenden Sie als Stichtag den 01.01.2021 (`comparison`)

In [ ]:
comparison = "2021-01-01"

In [ ]:
# Ihre Lösung

Lassen Sie sich anschließend das Eintrittsdatum und die Tage im Unternehmen von allen Angestellten anzeigen, die länger als 6000 Tage im Unternehmen sind.

In [ ]:
# Ihre Lösung

### Analyse

Lassen Sie sich die Departments des Datensatzes anzeigen (jedes nur einmal).

In [ ]:
# Ihre Lösung

Ermitteln Sie (wie eben bei der Arbeit mit RDDs) die Anzahl Angestellte pro Department.

In [ ]:
# Ihre Lösung

Geben Sie durchschnittliche Zufriedenheit aller Personen aus, die in der Abteilung *HR* arbeiten, sowie das Alter der ältesten Person der Abteilung.

In [ ]:
# Ihre Lösung

Wie erstellen nun ein weiteres DataFrame. Erstellen Sie aus den Daten in `dept_data` das DataFrame `departments` mit den angegebenen Spaltennamen. Geben Sie das erzeugte DataFrame aus.

In [ ]:
dept_data = [("Sales", "New York", 10001), 
            ("HR", "Los Angeles", 90001),
            ("Purchasing", "San Francisco", 94104),
            ("Marketing", "Philadelphia", 19110),
            ("Technology", "Los Angeles", 90099)]

column_names = ["name", "location", "zip_code"]

In [ ]:
# Ihre Lösung

Wie viele Angestellte arbeiten in Los Angeles?

In [ ]:
# Ihre Lösung

Lassen Sie sich die ersten 10 Gehälter der Angestellten ausgeben, die an der Ostküste arbeiten (in diesem Fall `zip_code < 90000`).

In [ ]:
# Ihre Lösung

### Caching
Durch die In-Memory-Verarbeitung werden DataFrames in der Regel nicht zwischengespeichert. Mit jeder Action werden alle Verarbeitungsschritte neu ausgeführt. Manchmal kann es aber sinnvoll sein, ein DataFrame zwischenzuspeichern (zu *cachen*). Beim Aufruf der ersten Action wird das DataFrame dann im Speicher gehalten.

(*Hinweis*: Gleiches gilt natürlich auch für RDDs, da DataFrames ja auf RDDs basieren)

Betrachten Sie den folgenden Code: An welcher Stelle könnte es sinnvoll sein, ein Caching durchzuführen und warum? Fügen Sie den Code für das Caching ein. Schreiben Sie in die untere Zelle Ihre Begründung und beschreiben Sie, welche DataFrames auf die gecachte Version zugreifen.

```python
df1 = spark.read.csv(path = "my_data.csv", header = True)
df2 = df1.filter(df1.salary.between(40000, 50000))
df3 = df2.select("name", "salary", "department", "satisfied")
df4 = df3.groupby("department").agg({"satisfied": "mean"})
df4.show()
df5 = df3.groupby("department").agg({"salary": "mean"})
df5.show()
df6 = df3.select("name").where(df3["satisfied"] == 1)
df6.show()
```

## SQL
Im Folgenden verwenden wir SQL zur Datenabfrage. Dafür erstellen wir zunächst eine temporale View, auf der dann die SQL-Abfragen erfolgen.

In [ ]:
spark.sql("\
CREATE TEMP VIEW employees \
USING CSV \
OPTIONS (path = 'employees_satisfaction_transformed.csv', header = 'True', inferSchema = 'True')")

Ermitteln Sie nun wieder die Anzahl der Angestellten pro Department, aber verwenden Sie dieses Mal SQL.

In [ ]:
# Ihre Lösung

## Ressourcen-Vebrauch

Sie arbeiten in diesen Übungsaufgaben mit ihrem eigenen Cluster. In der Praxis ist dies natürlich selten der Fall, hier werden meist viele Jobs gleichzeitig gestartet und konkurrieren um Ressourcen.

Öffnen Sie die __[WebUI von YARN](http://127.0.0.1:8088)__ und sehen sich die aktuellen Applikationen an. Sie sollten dort das aktuelle Jupyter Notebook finden. Solange die SparkSession (der SparkContext) erstellt ist, wird das Notebook dort als *RUNNING* aufgelistet. 

Wie viele Kerne und wie viel Speicher verbraucht die Spark-Applikation? Wie können Sie diese Summe berechnen?

*Hinweis: Werfen Sie einen Blick in die Datei `spark-defaults.conf` im `SPARK_CONF_DIR`.*

*Sollten Sie nicht wissen, wie Sie diese Datei finden können, finden Sie unter dem "Show Solution"-Button hilfreiche Kommandos.*

- Über "New" -> "Terminal" können Sie ein Terminal öffnen. Alternativ können Sie in einer Code-Zelle einen Bash-Befehl ausführen, indem Sie ein ! an den Anfang der Zelle schreiben
- Zugriff auf den Inhalt einer Umgebungs-Variablen mittels `$`, z.B. `$SPARK_CONF_DIR`
- Anzeigen von Dateiinhalten: `cat`

Stoppen Sie den SparkContext, um alle Ressourcen, die dieses Notebook benötigt, freizugeben.

In [ ]:
# Ihre Lösung

Öffnen Sie ein Terminal und starten Sie eine pyspark-Shell mit YARN als Ressourcen-Manager im client-mode mit 2 Exekutoren mit jeweils 1GB Memory und 2 Cores. Wie viele Cores und wie viel Speicher benötigt die Applikation und wie setzen sich diese Zahlen zusammen? Kontrollen Sie in der WebUI von YARN, ob Ihre Berechnungen korrekt sind.

Starten Sie anschließend in einem zweiten Terminal eine weitere pyspark-Shell (ohne die erste Shell zu beenden) mit YARN als Ressourcen-Manager im client-mode mit der gleichen Konfiguration (2 Exekutoren mit jeweils 1GB Memory und 2 Cores). Erstellen Sie dort ein RDD mit den Werten 1, 2, 3, 4 und lassen sich dieses mittels `collect()` ausgeben.

Was passiert und warum? Wie können Sie das Problem beheben?

Schließen Sie beide Shells und starten anschließend eine neue pyspark-Shell mit folgenden Eigenschaften:
- YARN als Ressourcen-Manager im client-mode
- 2 Exekutoren mit jeweils
    - 2GB Memory
    - 3 Cores
    
Wie viele Ressourcen vebraucht diese Applikation und wie setzen sich diese zusammen?

Starten Sie anschließend in einem zweiten Terminal noch eine pypsark-Shell (ohne die erste Shell zu beenden) mit folgendem Befehl:

`pyspark --master yarn --conf spark.yarn.am.cores=3`

Was passiert und warum? Überprüfen Sie auch den Status der Applikation in der WebUI von YARN.